In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, ConversationChain
import json

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0301')
original_chain = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [2]:
original_chain.run('what do you know about Python in less than 10 words')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: what do you know about Python in less than 10 words
AI:

> Finished chain.


'Python is a high-level programming language.'

In [6]:
extracted_messages = original_chain.memory.chat_memory.messages
extracted_messages

[HumanMessage(content='what do you know about Python in less than 10 words', additional_kwargs={}),
 AIMessage(content='Python is a high-level programming language.', additional_kwargs={})]

In [7]:
ingest_to_db = messages_to_dict(extracted_messages)
ingest_to_db

[{'type': 'human',
  'data': {'content': 'what do you know about Python in less than 10 words',
   'additional_kwargs': {}}},
 {'type': 'ai',
  'data': {'content': 'Python is a high-level programming language.',
   'additional_kwargs': {}}}]

In [8]:
retrieve_from_db = json.loads(json.dumps(ingest_to_db))
retrieve_from_db

[{'type': 'human',
  'data': {'content': 'what do you know about Python in less than 10 words',
   'additional_kwargs': {}}},
 {'type': 'ai',
  'data': {'content': 'Python is a high-level programming language.',
   'additional_kwargs': {}}}]

In [10]:
retrieved_messages = messages_from_dict(retrieve_from_db)
retrieved_messages

[HumanMessage(content='what do you know about Python in less than 10 words', additional_kwargs={}),
 AIMessage(content='Python is a high-level programming language.', additional_kwargs={})]

In [11]:
retrieved_chat_history = ChatMessageHistory(messages=retrieved_messages)
retrieved_chat_history

ChatMessageHistory(messages=[HumanMessage(content='what do you know about Python in less than 10 words', additional_kwargs={}), AIMessage(content='Python is a high-level programming language.', additional_kwargs={})])

In [12]:
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)
retrieved_memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what do you know about Python in less than 10 words', additional_kwargs={}), AIMessage(content='Python is a high-level programming language.', additional_kwargs={})]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history')

In [13]:
reloaded_chain = ConversationChain(
    llm=llm,
    verbose=True,
    memory=retrieved_memory
)
reloaded_chain.run('what about Javascript and Ruby')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: what do you know about Python in less than 10 words
AI: Python is a high-level programming language.
Human: what about Javascript and Ruby
AI:

> Finished chain.


'JavaScript is a scripting language used for web development, while Ruby is a dynamic, object-oriented programming language.'